In [ ]:
# use dream_proj_env conda environment
import pandas as pd
import numpy as np
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr, ttest_ind, mannwhitneyu,linregress
import gseapy
import scanpy as sc
from pybiomart import Dataset
import random
from pylr2 import regress2
import statsmodels.formula.api as smf
import colorcet as cc
# auto reload source files
%load_ext autoreload
%autoreload 2
# add source directory to path
source_path = os.path.abspath(os.path.join('..'))
if source_path not in sys.path:
    sys.path.append(os.path.join(source_path, 'source'))
# read source files
import read_data
from expr_dataset import ExpressionDataset
from meta_expr_dataset import MetaExpressionDataset

# repo root for relative paths
REPO_ROOT = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))

# colorpalettes 
hmm = ["#000000", "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7"]
fred_again_again_again_palette = ['#000305', '#ff7758', '#f67805', '#d3d3d3', '#565656', '#054fff', '#f9aa74', '#5890ff']
sunset = ['#f3e79b','#fac484','#f8a07e','#eb7f86','#ce6693','#a059a0','#5c53a5']
my_continous_palette = sunset
my_categorical_palette = []
my_categorical_palette.append(hmm[3])
my_categorical_palette.append(fred_again_again_again_palette[-2])
my_categorical_palette.append(hmm[-1])
my_categorical_palette.append(fred_again_again_again_palette[-1])

my_categorical_palette.append(hmm[2])
my_categorical_palette.append(sunset[-1])
my_categorical_palette.append(sunset[-2])
my_categorical_palette.append(sunset[-5])
my_categorical_palette.append(sunset[-7])

%config InlineBackend.figure_format = 'retina'
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
# create mapping of Organ to color
organ_palette = {
    "Liver": my_categorical_palette[1],
    "Kidney": my_categorical_palette[2],
    "Brain": my_categorical_palette[5],
}

# U2OS cells

In [3]:
loader = read_data.DatasetLoader("bujarrabal_dueso_2023_u2osRNA")
bujarrabal_dueso = loader.load_dataset()
bujarrabal_dueso.calc_total_seq_depth()
bujarrabal_dueso.get_dream_gene_expression()
bujarrabal_dueso.dream_enrichment_ssgsea()

Loading dataset: bujarrabal_dueso_2023_u2osRNA


Did not need to convert DREAM genes
Found 318 DREAM genes with expression
scaled mean_dream_reg_expr by sequence depth and created mean_dream_reg_expr_resid
scaled DREAM_normalized_enrichment_score by sequence depth and created DREAM_normalized_enrichment_score_resid
scaled DREAM_enrichment_score by sequence depth and created DREAM_enrichment_score_resid


In [ ]:
fig, axes = plt.subplots(figsize = (3, 4))
sns.stripplot(data = bujarrabal_dueso.dream_expression, x = 'treatment', y = 'DREAM_normalized_enrichment_score_resid', order = ['Harmine control', 'Harmine', 'INDY control', 'INDY'], palette = my_categorical_palette, ax = axes)
# add boxplot with no fill and my_categorical_palette edgecolor
sns.barplot(data = bujarrabal_dueso.dream_expression, x = 'treatment', y = 'DREAM_normalized_enrichment_score_resid',order = ['Harmine control', 'Harmine', 'INDY control', 'INDY'], palette = my_categorical_palette, 
            # white with black border
            linewidth = 1, edgecolor = 'black', ax = axes, facecolor = 'white', errcolor = 'black', errwidth=1.3, capsize = 0.2, errorbar= 'se')
sns.despine()
plt.ylabel('DREAM complex activity')
plt.xlabel('')
plt.ylim(0, 0.9)
# angle x tick labels
_ = plt.xticks(rotation = 15)

plt.savefig(os.path.join(REPO_ROOT, "figures/fig0/treated_dream_expression.svg"))
# do t-test for each treatment
ttest_res = {}
for treatment in ['Harmine control', 'Harmine', 'INDY control', 'INDY']:
    ttest_res[treatment] = ttest_ind(bujarrabal_dueso.dream_expression.query(f'treatment == "{treatment}"')['DREAM_normalized_enrichment_score_resid'], 
                                     bujarrabal_dueso.dream_expression.query(f'treatment == "{treatment} control"')['DREAM_normalized_enrichment_score_resid'])
    print(f"t-test for {treatment} vs {treatment} control: {ttest_res[treatment]}")

# LIN52 KO c. elegans

In [8]:
loader = read_data.DatasetLoader("bujarrabal_dueso_2023_wormRNA")
bujarrabal_dueso_worm = loader.load_dataset()
bujarrabal_dueso_worm.calc_total_seq_depth()
bujarrabal_dueso_worm.get_dream_gene_expression()
bujarrabal_dueso_worm.dream_enrichment_ssgsea()
bujarrabal_dueso_worm.dream_expression['genotype + treatment'] = bujarrabal_dueso_worm.dream_expression['genotype'] + ' ' + bujarrabal_dueso_worm.dream_expression['treatment']


Loading dataset: bujarrabal_dueso_2023_wormRNA


Converted DREAM genes to worm genes
Found 283 DREAM genes with expression
scaled mean_dream_reg_expr by sequence depth and created mean_dream_reg_expr_resid
scaled DREAM_normalized_enrichment_score by sequence depth and created DREAM_normalized_enrichment_score_resid
scaled DREAM_enrichment_score by sequence depth and created DREAM_enrichment_score_resid


In [ ]:
fig, axes = plt.subplots(figsize = (3, 4))
sns.stripplot(data = bujarrabal_dueso_worm.dream_expression, x = 'genotype + treatment', y = 'DREAM_normalized_enrichment_score_resid', palette = my_categorical_palette, ax = axes)
# add boxplot with no fill and my_categorical_palette edgecolor
sns.barplot(data = bujarrabal_dueso_worm.dream_expression, x = 'genotype + treatment', y = 'DREAM_normalized_enrichment_score_resid', palette = my_categorical_palette, 
            # white with black border
            linewidth = 1, edgecolor = 'black', ax = axes, facecolor = 'white', errcolor = 'black', errwidth=1.3, capsize = 0.2, errorbar= 'se')
# angle x tick lables
axes.set_xticklabels(axes.get_xticklabels(), rotation = 15, ha = 'right')
plt.ylabel('DREAM complex activity')
plt.xlabel('')
plt.ylim(0, 0.9)
sns.despine()

plt.savefig(os.path.join(REPO_ROOT, "figures/fig0/worm_WT_vs_LIN52KO_dream_expression.svg"))

# do t-test for each genotype + treatment
ttest_res = {}
wt = ['WT 6h post mock-UV-treatment', 'WT 6h post UV-treatment']
trts = [ 'lin-52(n771) 6h post mock-UV-treatment', 'lin-52(n771) 6h post UV-treatment']
for control_trt, trt in zip(wt, trts):
    ttest_res[treatment] = ttest_ind(bujarrabal_dueso_worm.dream_expression.loc[bujarrabal_dueso_worm.dream_expression['genotype + treatment'] == trt, 'DREAM_normalized_enrichment_score_resid'],
                                        bujarrabal_dueso_worm.dream_expression.loc[bujarrabal_dueso_worm.dream_expression['genotype + treatment'] == control_trt, 'DREAM_normalized_enrichment_score_resid'])
    print(f"t-test for {trt} vs {control_trt}: {ttest_res[treatment]}")

In [ ]:
fig, axes = plt.subplots(figsize = (2, 4))
sns.stripplot(data = bujarrabal_dueso_worm.dream_expression, x = 'genotype', y = 'DREAM_normalized_enrichment_score_resid', palette = my_categorical_palette[-4:], ax = axes)
# add boxplot with no fill and my_categorical_palette edgecolor
sns.barplot(data = bujarrabal_dueso_worm.dream_expression, x = 'genotype', y = 'DREAM_normalized_enrichment_score_resid', palette = my_categorical_palette[-4:], 
            # white with black border
            linewidth = 1, edgecolor = 'black', ax = axes, facecolor = 'white', errcolor = 'black', errwidth=1, capsize = 0.2, errorbar= 'se')
# angle x tick lables
sns.despine()
plt.ylabel('DREAM complex activity')
plt.xlabel('')
plt.ylim(0, 0.9)

plt.savefig(os.path.join(REPO_ROOT, "figures/fig0/worm_WT_vs_LIN52KO_dream_expression_genotype_only.svg"))

# do t-test for each genotype + treatment
ttest_res = {}
print(ttest_ind(bujarrabal_dueso_worm.dream_expression.loc[bujarrabal_dueso_worm.dream_expression['genotype'] == 'WT', 'DREAM_normalized_enrichment_score_resid'],
                                    bujarrabal_dueso_worm.dream_expression.loc[bujarrabal_dueso_worm.dream_expression['genotype'] == 'lin-52(n771)', 'DREAM_normalized_enrichment_score_resid']))